In [1]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import requests

In [2]:
Edges = pd.read_csv("citeseer Edges.csv",header=None)
Edges = np.array(Edges)

In [3]:
Labels = pd.read_csv("citeseer node_labels.csv",header=None)
y = np.array(Labels)

In [4]:
Edges.max(),np.shape(Edges),np.shape(y)

(3264, (4536, 2), (3264, 1))

In [5]:
A = np.zeros((3264,3264))
for i in range (0,4536):
    A[Edges[i,0]-1,Edges[i,1]-1]=1

In [6]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    end1 = time.time()
    newx=([])
    dmax=50
    tmp = np.zeros([dmax,1])
    for d in range (1,dmax):
        newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
        newx=np.reshape(newx,(n,d))
        clf2 = LinearDiscriminantAnalysis()
        newx_train, newx_test =newx[train_index], newx[test_index]
        clf2.fit(newx_train, y_train.ravel())
        tmp[d,0]=clf2.score(newx_test, y_test.ravel())
    db=tmp.argmax()
    ASELDAACC=tmp.max()
    ASELDATIME=end1-begin1
    #eigenvalues=s**2
    #Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*KNN
    begin2 = time.time()
    neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
    neigh.fit(z_train, y_train.ravel())
    AEEKNNACC = neigh.score(z_test, y_test)
    end2 = time.time()
    AEEKNNTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEKNNACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"d = {db}")
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [7]:
n=3264
k=int(y.max())
EncoderLDA(A)

d = 28
Accuracy for each method: [0.32515337 0.49539877 0.47392638]
Time for each method: [19.02563787  0.06806612  0.02360988]
